# 🚀 Flusso di Esecuzione per Training Qwen su Colab

**NOTA**: Questo notebook è configurato per un repository pubblico su GitHub.
Assicurati che il repository `Attapulgite999/prove` sia pubblico e contenga tutti i file necessari.

### 0. **Verifica Connessione Colab**

In [1]:
# Verifica connessione internet
!curl -s -o /dev/null -w "%{http_code}" https://www.google.com

# Verifica GPU
import torch
print("GPU disponibile:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Nome GPU:", torch.cuda.get_device_name(0))
    print("Memoria GPU totale:", torch.cuda.get_device_properties(0).total_memory // 1024**3, "GB")

# Verifica RAM
!free -h

# Verifica CPU
!nproc

200GPU disponibile: True
Nome GPU: Tesla T4
Memoria GPU totale: 14 GB
               total        used        free      shared  buff/cache   available
Mem:            12Gi       1.0Gi       1.1Gi       1.0Mi        10Gi        11Gi
Swap:             0B          0B          0B
2


### 1. **Preparazione Ambiente Colab**

In [2]:
# Monta Google Drive (OBBLIGATORIO se vuoi salvare il modello addestrato e il file GGUF)
from google.colab import drive
drive.mount('/content/drive')

# Clona il repository pubblico
!rm -rf prove

!git clone https://github.com/Attapulgite999/prove.git
%cd prove


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Cloning into 'prove'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 54 (delta 17), reused 50 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (54/54), 65.29 KiB | 8.16 MiB/s, done.
Resolving deltas: 100% (17/17), done.
/content/prove


### 2. **Installazione Dipendenze Base**

In [ ]:
# Aggiorna apt e installa python3 e pip (solitamente già presenti in Colab, ma non fa male)
!apt-get update && apt-get install -y python3 python3-pip

# Installa le dipendenze principali (assicurati che PyTorch sia compatibile con la tua versione CUDA)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -r requirements.txt

# Installa dipendenze aggiuntive se necessarie
!pip install transformers datasets accelerate peft

# FIX IMPORTANTE: Installa la versione corretta di trl richiesta da LLaMA Factory
!pip install ""

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

### 3. **Esecuzione Script Training**

In [ ]:
# Esegui lo script principale. Non devi specificare il percorso di Google Drive qui,
# lo script è già configurato per salvare l'output in `/content/drive/MyDrive/colab_training/medical_qwen_output`.
!python setup_and_train_colab.py

### 4. **Scarica il File GGUF (sul tuo computer locale)**

In [ ]:
# Il modello GGUF viene salvato automaticamente su Google Drive nella directory specificata.
# Questo comando serve per scaricarlo dal tuo ambiente Colab al tuo computer locale.
from google.colab import files
files.download('/content/drive/MyDrive/colab_training/medical_qwen_output/gguf_model/model.gguf')

## 🔧 Miglioramenti Implementati nello Script `setup_and_train_colab.py`

- **Retry automatico**: 3 tentativi per il caricamento del tokenizer e per il training.
- **Pre-check modello**: Verifica disponibilità del modello prima del training.
- **Gestione errori migliorata**: Log dettagliati per un troubleshooting più semplice.
- **Monitoraggio risorse**: Tracking dell'utilizzo di GPU memory e utilization.
- **Keep-alive**: Previene disconnessioni automatiche di Colab.
- **Resume da checkpoint**: Riprende il training dall'ultimo checkpoint in caso di interruzioni.
- **Download dataset automatico**: Scarica il dataset medico da Hugging Face se non presente localmente.
- **Merge LoRA e Conversione GGUF**: Unisce gli adapter LoRA al modello base e lo converte in formato GGUF per LM Studio.